In [1]:
import numpy as np
import csv
from scipy import stats


In [2]:
def read_and_preprocess_dataset (data_arq ):

    #leitura e pre processamento dos dados

    data_aux=[]
    with open(data_arq, newline='') as csvfile:
        readCSV = csv.reader(csvfile, delimiter=',')
        for row in readCSV:
            data_aux.append(row)
            
    
    data_aux = data_aux[1:]
 
        
    labels_aux = [ x[5] for i, x in enumerate(data_aux) ]
    labels_names = set(labels_aux)
    
    
    labels = []
    for label in labels_aux:
        labels.append([ i for i, x in enumerate(labels_names) if x==label ])
    labels = np.squeeze(np.array(labels))
    
    data = [ x[1:5] for i, x in enumerate(data_aux) ]
    data = np.array(data).astype(float)
    data = stats.zscore(data,axis=0)

    
    return data, labels


In [3]:
def folds_cross_val (data, labels, n_folds):
    
    n_np=np.sum(labels[:,0]==1)
    n_sp=np.sum(labels[:,1]==1)
    n_up=np.sum(labels[:,2]==1)
    
    
    vet_aux_np=np.arange(n_np)
    np.random.shuffle(vet_aux_np)
    
    vet_aux_sp=np.arange(n_sp) + n_np
    np.random.shuffle(vet_aux_sp)

    vet_aux_up=np.arange(n_up) + n_np + n_sp
    np.random.shuffle(vet_aux_up)

    dataset_folds={}
    labels_folds={}
    initial_np=0
    initial_sp=0
    initial_up=0
    for i in range (n_folds):
        
        if i<n_folds-1:
        
            aux_vet=np.concatenate( (vet_aux_np[initial_np : initial_np + round(len(vet_aux_np)/n_folds) ] , vet_aux_sp[initial_sp : initial_sp + round(len(vet_aux_sp)/n_folds) ]   , vet_aux_up[initial_up : initial_up + round(len(vet_aux_up)/n_folds) ])  , axis=0)   
            np.random.shuffle(aux_vet)       



            initial_np += round(len(vet_aux_np)/n_folds)
            initial_sp += round(len(vet_aux_sp)/n_folds)
            initial_up += round(len(vet_aux_up)/n_folds)

            dataset_folds[str(i)] = data[aux_vet,:]
            labels_folds[str(i)] = labels[aux_vet,:]
            
        else:
            
            aux_vet=np.concatenate( (vet_aux_np[initial_np : initial_np + round(len(vet_aux_np)/n_folds) ] , vet_aux_sp[initial_sp : initial_sp + round(len(vet_aux_sp)/n_folds) ]   , vet_aux_up[initial_up : initial_up + round(len(vet_aux_up)/n_folds) ])  , axis=0)   
            np.random.shuffle(aux_vet)       
          
                 
            dataset_folds[str(i)] = data[aux_vet,:]
            labels_folds[str(i)] = labels[aux_vet,:]
        
    
        
        
    return dataset_folds, labels_folds
        
        
        
        

In [4]:
def datasets_crossValitation (dataset_folds, labels_folds, i_crossVal):
    
    data_val=dataset_folds[str(i_crossVal)]
    label_val=labels_folds[str(i_crossVal)]
    
    data_test=dataset_folds[str(len(dataset_folds)-1)]
    label_test=labels_folds[str(len(dataset_folds)-1)]
            
    data_train=np.empty((0, data_val.shape[1]), int)
    label_train=np.empty((0, label_val.shape[1]), int)
    for i_b in range(len(dataset_folds)-1):
        if not(i_b==i_crossVal):
            data_train=np.append(data_train,dataset_folds[str(i_b)],axis=0)
            label_train=np.append(label_train,labels_folds[str(i_b)],axis=0)
    
    
    
            
    
        
    return data_train, data_val, data_test, label_train, label_val, label_test 

In [ ]:
def true_table(label_hat,label):

    label_hat=np.array(label_hat)

    y_hat=np.ones((label_hat.shape[0],label_hat.shape[1]))*(-1)
    arg_max=np.argmax(label_hat,axis=1)
    for i in range(label_hat.shape[0]):
        y_hat[i,arg_max[i]]=1

    tt=np.zeros((label.shape[1],label_hat.shape[1]))
    for l in range(label.shape[1]):
        for c in range(label_hat.shape[1]):
            tt[l][c]=100*sum((y_hat[:,c]==1) & (label[:,l]==1))/sum(label[:,l]==1)
        
        
        
    return tt

In [ ]:
def true_table_kMeans(label_hat,label,cs):

    label_hat=np.array(label_hat)

    
    tt=np.zeros((label.shape[1],len(cs)))
    for l in range(label.shape[1]):
        for c in range(len(cs)):
            tt[l][c]=sum((label_hat==cs[c]) & (label[:,l]==1))
        
        
        
    return tt